In [1]:
import SimPEG as simpeg
import simpegEM as simpegem, simpegMT as simpegmt
from SimPEG.Utils import meshTensor
import numpy as np
import simpegMT.Tests.test_Problem3D_againstAnalytic as t3Dmt

In [2]:
sigmaHalf = 0.01
survey, problem = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.halfSpace(sigmaHalf),comp='zxyi',singleFreq=1)
if False:
    fields = problem.fields(problem.curModel.sigma)
    data = survey.dpred(problem.curModel.sigma)

In [3]:
def testDeriv_dA_dm(prb,cond):
    TOL = 1e-4
    FLR = 1e-20
    x0 = np.log(np.ones(prb.mesh.nC)*cond)
    prb.mapping = simpeg.Maps.ExpMap(prb.mesh)
    if True:
        x0  = x0 + np.random.randn(prb.mesh.nC)*cond*1e-1 
    survey = prb.survey
    src = survey.srcList[0]
    freq = src.freq
    v1 = np.random.randn(prb.mesh.nE,1)
    v2 = np.random.randn(prb.mesh.nE,1)
    v = np.hstack(( simpeg.mkvc(v1,2), simpeg.mkvc(v2,2)))
    u_0 = prb.fieldsPair(prb.mesh,prb.survey)
    u_0[src,'e_pxSolution'] = v1
    u_0[src,'e_pySolution'] = v2
    def fun(x):
        prb.curModel = x
        A = prb.getA(freq) #
#         return simpeg.mkvc(A*v1)+simpeg.mkvc(A*v2), lambda t: simpeg.mkvc(prb.getADeriv_m(freq, u_0[src], t))
        return A*v, lambda t: (prb.getADeriv_m(freq, u_0[src], t))
    return simpeg.Tests.checkDerivative(fun, x0, num=3, plotIt=False, eps=FLR)
testDeriv_dA_dm(problem,0.1)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    3.058e+11     3.238e+10      nan
 1   1.00e-02    3.096e+10     3.246e+08      1.999
 2   1.00e-03    3.102e+09     3.251e+06      1.999
========================= PASS! =========================
Happy little convergence test!



True

In [4]:
def testDeriv_dRHS_dm(prb,cond):
    TOL = 1e-4
    FLR = 1e-20
    x0 = np.log(np.ones(prb.mesh.nC)*cond)
    prb.mapping = simpeg.Maps.ExpMap(prb.mesh)
    if True:
        x0  = x0 + np.random.randn(prb.mesh.nC)*cond*1e-1 
    survey = prb.survey
    src = survey.srcList[0]
    
    u0 = prb.fields(x0)
    freq = src.freq
    A = prb.getA(freq) #
    A_I = prb.Solver(A, **prb.solverOpts)

    ftype = prb._fieldType + 'Solution'
    u0_src = u0[src, ftype]
    v = np.random.randn(prb.mesh.nE,1)
    
    def fun(x):
        prb.curModel = x
        return simpeg.mkvc(np.sum(prb.getRHS(freq))), lambda x: simpeg.mkvc(prb.getRHSDeriv_m(freq, x))
#         return simpeg.mkvc(prb.fields(x)[src,prb._fieldType + 'Solution']), lambda x: simpeg.mkvc(prb.getADeriv_m(freq, u0_src, x))
    return simpeg.Tests.checkDerivative(fun, x0, num=3, plotIt=False, eps=FLR)
testDeriv_dA_dm(problem,0.1)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    3.151e+11     3.512e+10      nan
 1   1.00e-02    3.103e+10     3.483e+08      2.004
 2   1.00e-03    3.101e+09     3.486e+06      2.000
========================= PASS! =========================
Yay passed!



True

In [5]:
def testDeriv_S_e(prb,cond):
    # Initate things for the derivs Test
    TOL = 1e-4
    FLR = 1e-20
    
    src = prb.survey.srcList[0]
    rx = src.rxList[0]

    x0 = np.log(np.ones(prb.mesh.nC)*cond)
#     prb.mapping = simpeg.Maps.ExpMap(prb.mesh)
    if True:
        x0  = x0 + np.random.randn(prb.mesh.nC)*cond*1e-1 
    def fun(x):
        prb.curModel = x
        return src.S_e(prb), lambda t: src.S_eDeriv_m(prb,t)
    simpeg.Tests.checkDerivative(fun,x0,num=3,plotIt=False)
survey, problem = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.random(1),comp='All',singleFreq=1)
testDeriv_S_e(problem,0.1)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    2.848e+10     2.846e+09      nan
 1   1.00e-02    2.837e+09     2.801e+07      2.007
 2   1.00e-03    2.838e+08     2.800e+05      2.000
========================= PASS! =========================
That was easy!



In [6]:
src = survey.srcList[0]
u = problem.fields(problem.curModel)
u_src = u[src,:]
w = np.random.rand(problem.mesh.nC)


In [7]:
dA_dm = problem.getADeriv_m(src.freq,u_src,w)

In [8]:
dA_dm.shape

(20536, 2)

In [9]:
a, b, sig, d, e = t3Dmt.halfSpace(.01)

In [21]:
def testDeriv_ProjFields(prb):
    # Initate things for the derivs Test
    src = survey.srcList[0]
    rx = src.rxList[0]
    rx.locs = np.array([[0.,0.,0,],[1.,1.,1.]])
    u0x = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
    u0y = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
#     u0 = np.vstack((simpeg.mkvc(u0x,2),simpeg.mkvc(u0y,2)))
    u0 = np.r_[u0x, u0y]
    f0 = prb.fieldsPair(survey.mesh,survey)
    # u0 = np.hstack((simpeg.mkvc(u0_px,2),simpeg.mkvc(u0_py,2)))
    f0[src,'e_pxSolution'] =  u0[:len(u0)/2]#u0x
    f0[src,'e_pySolution'] = u0[len(u0)/2::]#u0y
    # f0[src,'b_1d'] = -1/(1j*simpegem.Utils.EMUtils.omega(src.freq))*m1d.nodalGrad*u0
    # Run a test
    def fun(u):
        f = prb.fieldsPair(survey.mesh,survey)
        f[src,'e_pxSolution'] = u[:len(u)/2]
        f[src,'e_pySolution'] = u[len(u)/2::]
    #     f[src,'b_1d'] = -(m1d.nodalGrad*u)/(1j*simpegem.Utils.EMUtils.omega(src.freq))
        return rx.projectFields(src,survey.mesh,f), lambda t: rx.projectFieldsDeriv(src,survey.mesh,f0,t)
    simpeg.Tests.checkDerivative(fun,u0,num=8,plotIt=False)
survey, problem = t3Dmt.setupSimpegMTfwd_eForm_ps(t3Dmt.random(.1),comp='zyxr',singleFreq=.01)
testDeriv_ProjFields(problem)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    3.699e-07     6.864e-07      nan
 1   1.00e-02    3.693e-08     6.856e-08      1.001
 2   1.00e-03    3.693e-09     6.855e-09      1.000
 3   1.00e-04    3.693e-10     6.855e-10      1.000
 4   1.00e-05    3.693e-11     6.855e-11      1.000
 5   1.00e-06    3.693e-12     6.855e-12      1.000
 6   1.00e-07    3.693e-13     6.855e-13      1.000
 7   1.00e-08    3.693e-14     6.855e-14      1.000
*********************************************************
<<<<<<<<<<<<<<<<<<<<<<<<< FAIL! >>>>>>>>>>>>>>>>>>>>>>>>>
*********************************************************
You had so much promise Gudni, oh well...



In [11]:
src = survey.srcList[0]
rx = src.rxList[0]

In [12]:
rx.locs = np.array([[0, 0, 0],[1,1,1]])

In [14]:
u0x = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j
u0y = np.random.randn(survey.mesh.nE)+np.random.randn(survey.mesh.nE)*1j    
src = survey.srcList[0]
rx = src.rxList[0]
f0 = problem.fieldsPair(survey.mesh,survey)
u0 = np.vstack((simpeg.mkvc(u0x,2),simpeg.mkvc(u0y,2)))
f0[src,'e_pxSolution'] = u0x
f0[src,'e_pySolution'] = u0y

In [17]:
rx.projectFieldsDeriv(src,survey.mesh,f0,u0).shape

(2, 1)

In [16]:
simpeg.Utils.spzeros?

In [ ]:
rx.projectFields(src,survey.mesh,f0)

In [ ]:
%debug